In [4]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

def knnVote():
    '''
    KNN using majority vote
    '''
    #nearest indices
    A = tf.constant([1, 1, 2, 4, 4, 4, 7, 8, 8])
    A = tf.constant([1, 1, 2, 4, 4, 4, 7, 8, 8])
    print(A.shape)

    nearest_k_y, idx, votes = tf.unique_with_counts(A)
    print("y", nearest_k_y.eval())
    print("idx", idx.eval())
    print("votes", votes.eval())
    majority = tf.argmax(votes)
    predict_res = tf.gather(nearest_k_y, majority)
    
    
    print("majority", majority.eval())
    print("predict", predict_res.eval())
    return predict_res

def err():
    #nearest indices
    A = tf.constant([1, 1, 2, 4, 4, 4, 7, 8, 8])
    B = tf.constant([1, 1, 2, 4, 4, 4, 7, 8, 8])
    loss = tf.equal(A, B )
    print(loss.eval())

init = tf.global_variables_initializer()
sessMain = tf.InteractiveSession()
sessMain.run(init)
err()



[ True  True  True  True  True  True  True  True  True]


In [8]:
#def multidimensionalKnnVote():
A = tf.constant([
    [1, 1, 2, 4, 4, 4, 7, 8, 8],
    [2, 2, 3, 3, 3, 4, 4, 5, 6],
  ])

def cond(i, all_idxs, all_vals):
    return i < A.shape[0]

def body(i, all_idxs, all_vals):
    nearest_k_y, idx, votes = tf.unique_with_counts(A[i])
    majority_idx = tf.argmax(votes)
    majority_val = nearest_k_y[majority_idx]

    majority_idx = tf.reshape(majority_idx, shape=(1,))
    majority_val = tf.reshape(majority_val, shape=(1,))

    new_idxs = tf.cond(tf.equal(i, 0),
                       lambda: majority_idx,
                       lambda: tf.concat([all_idxs, majority_idx], axis=0))

    new_vals = tf.cond(tf.equal(i, 0),
                       lambda: majority_val,
                       lambda: tf.concat([all_vals, majority_val], axis=0))

    return i + 1, new_idxs, new_vals

# This means: starting from 0, apply the `body`, while the `cond` is true.
# Note that `shape_invariants` allow the 2nd and 3rd tensors to grow.
i0 = tf.constant(0)
idx0 = tf.constant(0, shape=(1,), dtype=tf.int64)
val0 = tf.constant(0, shape=(1,), dtype=tf.int32)
_, idxs, vals = tf.while_loop(cond, body,
                                loop_vars=(i0, idx0, val0),
                                shape_invariants=(i0.shape, tf.TensorShape([None]), tf.TensorShape([None])))

print('majority:', idxs.eval())
print('predict:', vals.eval())

majority: [2 1]
predict: [4 3]
